In [ ]:
import numpy as np
import os
import pickle
import random

import import_ipynb
from utils import load_best_checkpoint, normalize, cosine_similarity
from constants import NUM_FBANK, HOP_LENGTH, NUM_SPEC_PER_SPEAKER, NUM_FRAME, NUM_ENROLL_UTT, NUM_TRUE_UTT, NUM_FALSE_UTT
from eval_metrics import calculate_eer

In [ ]:
def test_frame(model):
  print('============================================')
  print('start frame level test')

  # load data
  val_dataset_path = '../data/dataset/test_340_25_128_512.npy'
  with open(val_dataset_path,"rb") as f: batchs = pickle.load(f)
  
  num_batch = batchs.shape[0]
  BATCH_SIZE = NUM_ENROLL_UTT + NUM_TRUE_UTT + NUM_FALSE_UTT
  true_score = []   # true 발화의 점수를 저장할 배열
  false_score = []  # false 발화의 점수를 저장할 배열

  batchs = np.concatenate(batchs)
  pred = model.predict(batchs)
  pred = normalize(pred)
  del batchs

  for i in range(num_batch):
    enroll = pred[i*BATCH_SIZE:i*BATCH_SIZE+NUM_ENROLL_UTT]
    enroll = normalize(np.mean(enroll, axis=0)) # 등록 발화를 평균

    for j in range(NUM_TRUE_UTT): true_score.append(cosine_similarity(enroll,pred[i*BATCH_SIZE+NUM_ENROLL_UTT+j]))
    for j in range(NUM_FALSE_UTT): false_score.append(cosine_similarity(enroll,pred[i*BATCH_SIZE+NUM_ENROLL_UTT+NUM_TRUE_UTT+j]))
  
  # calculate_eer함수의 인자로 알맞은 형태로 변환. true의 label에 1 표시해주기
  scores = np.array(true_score+false_score)
  labels = np.array([1.0]*len(true_score)+[0.0]*len(false_score))

  # eer 계산
  eer, thres = calculate_eer(np.arange(0, 1.0, 0.001), scores, labels, get_threshold=True)
  print('EER:', eer, '  thres:',thres)

In [ ]:
def split_spec(spec):
  splited = []
  for i in range(0,spec.shape[0]-NUM_FRAME+1,NUM_FRAME//2): splited.append(spec[i:i+NUM_FRAME,:])
  return np.array(splited)

def test_utt(model):
  print('==================================================')
  print('start utterence level test')

  # load data
  dataset_path = '../data/dataset/testUtt_201_25.pickle'
  with open(dataset_path,"rb") as f: batchs = pickle.load(f)
  
  true_score = []   # true 발화의 점수를 저장할 배열
  false_score = []  # false 발화의 점수를 저장할 배열

  for batch in batchs:
    d_vectors = []
    for utt in batch:
      specs = split_spec(utt)
      pred = model.predict(specs)
      pred = normalize(pred)
      d_vector = normalize(np.mean(pred, axis=0))
      d_vectors.append(d_vector)
    
    enroll = d_vectors[:NUM_ENROLL_UTT]
    enroll = normalize(np.mean(enroll, axis=0)) # 등록 발화를 평균

    for j in range(NUM_TRUE_UTT): true_score.append(cosine_similarity(enroll,d_vectors[NUM_ENROLL_UTT+j]))
    for j in range(NUM_FALSE_UTT): false_score.append(cosine_similarity(enroll,d_vectors[NUM_ENROLL_UTT+NUM_TRUE_UTT+j]))
  
  # calculate_eer함수의 인자로 알맞은 형태로 변환. true의 label에 1 표시해주기
  scores = np.array(true_score+false_score)
  labels = np.array([1.0]*len(true_score)+[0.0]*len(false_score))

  # eer 계산
  eer, thres = calculate_eer(np.arange(0, 1.0, 0.001), scores, labels, get_threshold=True)
  print('EER:', eer, '  thres:',thres)

In [ ]:
#generate_val_dataset('C:/Users/LeeJunghun/Desktop/lingometer/data/NIKL_DIALOGUE_2020_PCM_v1.2_part1/NIKL_DIALOGUE_2020_PCM_v1.2_part1/npy/test', 'C:/Users/LeeJunghun/Desktop/lingometer/data/NIKL_DIALOGUE_2020_PCM_v1.2_part1/NIKL_DIALOGUE_2020_PCM_v1.2_part1/test_frame_batcher')